###### excel -> csv 변환

In [10]:
import pandas as pd
import numpy as np
import os

In [3]:
pwd

'C:\\MarketCaster\\Industrial Index'

In [4]:
# 년, 월, 일로 표기된 DATE를 처리하기 위한 function
# input 인자인 dataFrame은 AMD['DATE']와 같이 dataframe 내 컬럼을 명시해주어야 함
# 2019년 12월 4일 -> 2019-12-04
def date_preprocessing(dataFrame):
    i = 0
    for date in dataFrame:
        try:
            # 띄어쓰기 제거
            tmp1, tmp2, tmp3 = date.split(' ')
            # 연, 월, 일 조립
            tmp = tmp1[:4] + '-' + tmp2[:2] + '-' + tmp3[:2]
        except:
            tmp = date
        # reset
        dataFrame.loc[i] = pd.to_datetime(tmp)
        i = i + 1

In [5]:
# 거래량이 M(Milion) 단위로 표기된 VOLUME을 처리하기 위한 function
# input 인자인 dataFrame은 AMD['VOLUME']와 같이 dataframe 내 컬럼을 명시해주어야 함
# 10.00M -> 10000000
def volume_preprocessing(dataFrame):
    i = 0
    for volume in dataFrame:
        if type(volume) == str:
            if volume[-1] == 'M':
                volume = int(float(volume[:-1]) * 1000000)
            elif volume == '-':
                volume = 0
            dataFrame.loc[i] = int(volume)
        elif volume == np.nan:
            pass
        else:
            try:
                dataFrame.loc[i] = int(dataFrame.loc[i])
            except:
                pass
        i = i + 1

In [6]:
# 일별수익률에 %기호가 붙어있는 경우 사용
def change_preprocessing(dataFrame):
    i = 0
    for change in dataFrame:
        try:
            # %기호 제거
            if change[-1] == '%':
                change = change[:-1]
        except:
            pass
        try:
            dataFrame.loc[i] = float(change)
        except:
            print(type(change), ' ', i)
        i = i + 1

In [7]:
# 사용할 데이터의 기간 설정
# date_from < 사용할 기간 < date_to
date_from = pd.to_datetime('2006-01-01')
date_to   = pd.to_datetime('2019-11-01')

# investing.com 데이터 중 비영업일이 영업일로 체크된 날짜 설정
investing_ignore_date = pd.to_datetime('2016-02-27')

In [8]:
# data file이 위치한 directory 설정
path = './Data/US/'
csv_path = './Data/CSV/'
# 최종 data name 설정
describe_file_name = 'describe_US.csv'

In [11]:
# directory 내 전체 file list 추출
file_list = os.listdir(path)
file_list

['AMD.csv',
 'APPLE.csv',
 'Intel.csv',
 'ISECloudComputingIndex.xlsx',
 'Micron.csv',
 'Nvidia.csv',
 'Original',
 'Philadelphia.xlsx',
 'QCOM.csv']

In [12]:
# csv file 추출
file_list_csv = [file for file in file_list if file.endswith(".csv")]
file_list_csv

['AMD.csv', 'APPLE.csv', 'Intel.csv', 'Micron.csv', 'Nvidia.csv', 'QCOM.csv']

In [13]:
# xlsx file 추출
file_list_xlsx = [file for file in file_list if file.endswith(".xlsx")]
file_list_xlsx

['ISECloudComputingIndex.xlsx', 'Philadelphia.xlsx']

In [14]:
# 파일명을 저장하기 위한 list 선언
df_name_list = []
# dataFrame을 저장하기 위한 list 선언
df_list = []

In [15]:
# directory 내 csv file read
for file in file_list_csv:
    df_list.append(pd.read_csv(path+file, encoding='CP949'))
    df_name_list.append(file[:-4])

In [16]:
# directory 내 xlsx file read
for file in file_list_xlsx:
    df_list.append(pd.read_excel(path+file, sheet_name='Sheet1'))
    df_name_list.append(file[:-5])

In [17]:
df_name_list

['AMD',
 'APPLE',
 'Intel',
 'Micron',
 'Nvidia',
 'QCOM',
 'ISECloudComputingIndex',
 'Philadelphia']

In [18]:
len(df_list)

8

In [19]:
investing_ignore_date = pd.to_datetime('2016-02-27')
tmp_df_list = []
for df in df_list:
    date_preprocessing(df['DATE'])
    volume_preprocessing(df['VOLUME'])
    change_preprocessing(df['CHANGE'])
    df['CHANGE'] = df['CHANGE'].apply(pd.to_numeric)
    tmp_df = df[df['DATE'] < date_to]
    
    # investing.com에서 영업일로 표기된 날짜에 대한 index 추출 및 행 삭제
    try:
        index = df[df['DATE'] == investing_ignore_date].index[0]
        tmp_df = tmp_df.drop(index, axis = 0)
        tmp_df_list.append(tmp_df)
    except:
        tmp_df_list.append(tmp_df)
    

C:\Users\Seungwon\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


-------

In [47]:
describe_df_list = pd.DataFrame(columns=column_list[1:])
i = 0
for df in tmp_df_list:
    date_data = pd.DataFrame([[df_name_list[i], df['DATE'].min(), df['DATE'].max(), len(df['DATE']), np.nan, np.nan]],
                             columns=column_list[1:])
    describe_df_list = pd.concat([describe_df_list, date_data], axis = 0, sort=False)
    describe_df_list = pd.concat([describe_df_list, df.describe()], axis = 0, sort=False)
    i += 1

In [50]:
describe_df_list.to_csv('describe_US_test.csv')

In [27]:
tmp_df_list[7]

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,CHANGE
21,2019-10-31,1652.888,1653.213,1630.866,1651.225,50286736.0,-0.616
22,2019-10-30,1670.927,1670.965,1649.676,1661.460,44595396.0,-0.060
23,2019-10-29,1682.122,1691.162,1661.146,1662.460,48164208.0,-0.901
24,2019-10-28,1662.638,1679.727,1662.638,1677.569,46648084.0,1.753
25,2019-10-25,1624.674,1650.199,1624.591,1648.671,51457184.0,2.053
...,...,...,...,...,...,...,...
3498,2006-01-09,520.830,525.830,518.850,524.330,85132528.0,0.916
3499,2006-01-06,510.100,520.430,508.870,519.570,101875800.0,2.169
3500,2006-01-05,497.880,509.540,497.880,508.540,99354176.0,2.386
3501,2006-01-04,493.330,498.940,493.330,496.690,87057856.0,0.722


In [30]:
# csv file save
i = 0
for df in tmp_df_list:
    df.to_csv(csv_path+df_name_list[i]+'.csv', index=False)
    i += 1